# Register TSV Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

Now that we have a database, we’re ready to create a table that’s based on the `Amazon Customer Reviews Dataset`. We define the columns that map to the data, specify how the data is delimited, and provide the location in Amazon S3 for the file(s). 


<img src="img/athena_register_tsv.png" width="60%" align="left">

In [4]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [5]:
ingest_create_athena_table_csv_passed = False

In [6]:
%store -r ingest_create_athena_db_passed

In [7]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(ingest_create_athena_db_passed)

True


In [9]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [10]:
%store -r s3_private_path_csv

In [11]:
try:
    s3_private_path_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [12]:
#print(s3_private_path_csv)
#s3_private_path_csv = "s3://{}/amazon-reviews-pds/tsv".format(bucket)
s3_private_path_csv = "s3://sagemaker-studio-898900188658-v9o4cft4fj/gun-violence"
s3_test_csv = "s3://sagemaker-studio-898900188658-v9o4cft4fj/test-folder/"
#/test2.csv

print(s3_private_path_csv)
%store s3_private_path_csv

s3://sagemaker-studio-898900188658-v9o4cft4fj/gun-violence
Stored 's3_private_path_csv' (str)


# Import PyAthena

In [13]:
from pyathena import connect

# Create Athena Table from Local CSV Files

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [14]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [15]:
# Set Athena parameters
database_name = "dsoaws"
table_name_csv = "gun_violence_csv"
test_table_name_csv = "test_table4"

In [16]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [36]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         incident_id varchar(10),
         incident_date date,
         state varchar(25),
         city_or_county varchar(25),
         address string,
         n_killed int,
         n_injured int,
         incident_url string,
         source_url string,
         incident_url_fields_missing string,
         congressional_district int,
         gun_stolen string,
         gun_type string,
         incident_characteristics string,
         latitude float,
         location_description string,
         longitude float,
         n_guns_involved int,
         notes string,
         participant_age string,
         participant_age_group string,
         participant_gender string,
         participant_name string,
         participant_relationship string,
         participant_status string,
         participant_type string,
         sources string,
         state_house_district int,
         state_senate_district int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)


print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.gun_violence_csv2(
         incident_id varchar(10),
         incident_date date,
         state varchar(25),
         city_or_county varchar(25),
         address string,
         n_killed int,
         n_injured int,
         incident_url string,
         source_url string,
         incident_url_fields_missing string,
         congressional_district int,
         gun_stolen string,
         gun_type string,
         incident_characteristics string,
         latitude float,
         location_description string,
         longitude float,
         n_guns_involved int,
         notes string,
         participant_age string,
         participant_age_group string,
         participant_gender string,
         participant_name string,
         participant_relationship string,
         participant_status string,
         participant_type string,
         sources string,
         state_house_district int,
         state_senate_district int
) ROW FORMA

In [18]:
# SQL statement to execute
#statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
#         incident_id string,
#         stuff string
#         ) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
#""".format(
#    database_name, test_table_name_csv, s3_test_csv
#)
#

#print(statement)

In [37]:
import pandas as pd

pd.read_sql(statement, conn)

""


# Verify The Table Has Been Created Succesfully

In [38]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,gun_violence_csv
1,gun_violence_csv2
2,test_table
3,test_table2
4,test_table3


In [39]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_tsv_passed = True

In [40]:
%store ingest_create_athena_table_csv_passed

Stored 'ingest_create_athena_table_csv_passed' (bool)


# Run A Sample Query

In [41]:
product_category = "California"

statement = """SELECT * FROM {}.{}
    WHERE state = '{}' LIMIT 100""".format(
    database_name, table_name_csv, product_category
)

print(statement)

SELECT * FROM dsoaws.gun_violence_csv2
    WHERE state = 'California' LIMIT 100


In [42]:
#statement = """SELECT * FROM {}.{}
#""".format(
#    database_name, test_table_name_csv
#)

#print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

,incident_id,incident_date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,-118.333,,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,,NaN,NaN
1,479389,2013-01-21,California,Brentwood,1100 block of Breton Drive,0,4,http://www.gunviolencearchive.org/incident/479389,http://sanfrancisco.cbslocal.com/2013/01/22/4-...,False,...,,Perps were likely motivated by gang affliations,,0::Teen 12-17||1::Teen 12-17||2::Teen 12-17||4...,0::Male||1::Male||2::Male||3::Male||4::Male,,,0::Injured||1::Injured||2::Injured||3::Injured...,NaN,NaN
2,479580,2013-02-03,California,Yuba (county),5800 block of Poplar Avenue,1,3,http://www.gunviolencearchive.org/incident/479580,http://sacramento.cbslocal.com/2013/02/04/1-de...,False,...,,-121.583,1,"""perps have gang affiliation",but unclear as to why victims were targeted. ...,ex-felon,"found w/ weaps""",0::20||4::25||5::18||6::19,NaN,NaN
3,480311,2013-02-11,California,Vallejo,800 block of Humboldt Street,1,4,http://www.gunviolencearchive.org/incident/480311,http://archive.news10.net/news/article/229997/...,False,...,-122.228,,,0::22,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Female||4::Female,0::Oscar Garcia,,NaN,NaN
4,480358,2013-02-19,California,Orange (county),Katella Avenue,4,3,http://www.gunviolencearchive.org/incident/480358,http://www.dailymail.co.uk/news/article-228117...,False,...,33.8031,,-117.943,1,Aoki killed in Ladera Ranch; 3 vics shot while...,0::20||4::69||5::27||6::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||4::Male||5::Male||6::Male,NaN,NaN


In [ ]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

# Review the New Athena Table in the Glue Catalog

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

# Store Variables for the Next Notebooks

In [ ]:
%store

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}